In [3]:
import sqlalchemy
import pandas as pd
from binance.client import Client

In [4]:
%run ../Binance_keys.ipynb

In [6]:
client = Client(api_key,api_secret)

In [7]:
engine = sqlalchemy.create_engine('sqlite:///BTCBUSDstream.db')

In [8]:
#Trendfollowing
#if the crypto was rising by 0.15% -> Buy
#exit when profit is above 0.15% or loss is crossing -0.15%

In [9]:
def strategy(entry, lookback, qty, open_position=False):
    while True:
        df = pd.read_sql('BTCBUSD', engine)
        lookbackperiod = df.iloc[-lookback:]
        cumret = (lookbackperiod.Price.pct_change() +1).cumprod() - 1
        if not open_position:
            if cumret[cumret.last_valid_index()] > entry:
                order = client.create_order(symbol='BTCBUSD',
                                           side='BUY',
                                           type='MARKET',
                                           quantity=qty)
                print(order)
                open_position = True
                break
    if open_position:
        while True:
            df = pd.read_sql('BTCBUSD', engine)
            sincebuy = df.loc[df.Time > 
                              pd.to_datetime(order['transactTime'],
                                            unit='ms')]
            if len(sincebuy) > 1:
                sincebuyret = (sincebuy.Price.pct_change() +1).cumprod() - 1
                last_entry = sincebuyret[sincebuyret.last_valid_index()]
                if last_entry > 0.0015 or last_entry < -0.0015:
                    order = client.create_order(symbol=pair,
                                           side='SELL',
                                           type='MARKET',
                                           quantity=qty)
                    print(order)
                    break

In [ ]:
strategy(0.001, 60, 0.001)